<a href="https://colab.research.google.com/github/seirah-yang/Colabwork/blob/main/%EC%BD%94%EC%8A%A4%EB%8B%A5_%EC%BD%94%EC%8A%A4%ED%94%BC_%EB%82%98%EC%8A%A4%EB%8B%A5_S%26P500_%EB%B0%B0%EB%8B%B9%EC%A0%95%EB%B3%B4_from_yahoofinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##실행하면 만들어지는 파일

./data/dividends_all_companies_5y.csv : 일자별 배당 (티커, 이름, 날짜, 주당배당금)

./data/dividends_yearly_sum.csv : 연도별 합계 배당

./data/tickers_not_found.csv : 야후에 존재하지 않는 티커

./data/tickers_with_no_dividends.csv : 최근 5년 배당 기록이 없는 유효 티커

In [10]:
# ─────────────────────────────────────────────────────────────────────────────
#  야후 파이낸스 존재 종목만 대상으로 최근 5년 배당금 수집 (국내+해외)
#  - 국내: KOSPI/KOSDAQ 전 종목 자동 생성 (KIND 다운로드 테이블 활용)
#  - 해외: S&P 500, NASDAQ-100 위키 테이블 자동 생성 + EXTRA_TICKERS로 임의 추가 가능
#  출력: ./data/dividends_all_companies_5y.csv, dividends_yearly_sum.csv, 기타 보조 CSV
# ─────────────────────────────────────────────────────────────────────────────
!pip -q install yfinance pandas lxml tqdm

In [57]:
import os, time, re, math, traceback
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [58]:
# ====== 0) 설정 ======
OUT_DIR = "./data"
os.makedirs(OUT_DIR, exist_ok=True)

DIV_CSV       = os.path.join(OUT_DIR, "dividends_all_companies_5y.csv")
YEARLY_CSV    = os.path.join(OUT_DIR, "dividends_yearly_sum.csv")
NOT_FOUND_CSV = os.path.join(OUT_DIR, "tickers_not_found.csv")
NO_DIV_CSV    = os.path.join(OUT_DIR, "tickers_with_no_dividends.csv")


In [65]:
LAST_N_YEARS       = 5
START_DATE         = (datetime.now() - timedelta(days=365*LAST_N_YEARS)).strftime("%Y-%m-%d")
MAX_WORKERS_CHECK  = 8     # 야후 존재성 확인 parallel
MAX_WORKERS_DIV    = 8     # 배당 수집 parallel
REQUEST_SLEEP      = 0.05  # 요청 간 짧은 지연 (야후 rate limit 고려)
RETRY              = 3

INCLUDE_KRX        = True  # 국내(KOSPI/KOSDAQ) 포함
INCLUDE_SP500      = True  # S&P500 포함
INCLUDE_NASDAQ100  = True  # NASDAQ-100 포함
EXTRA_TICKERS      = []    # 해외 임의 추가 티커 예: ['TSLA','NVDA','7203.T','005930.KS']


In [66]:
def build_universe_krx() -> pd.DataFrame:
    """
    KRX KIND 다운로드 테이블을 읽어 KOSPI/KOSDAQ 전 종목 생성
    """
    # KOSPI
    url_kospi = "http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13"
    df_kospi = pd.read_html(url_kospi, header=0, encoding="euc-kr")[0]  # euc-kr 지정
    df_kospi["시장구분"] = "KOSPI"

    # KOSDAQ
    url_kosdaq = "http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&marketType=kosdaq&searchType=13"
    df_kosdaq = pd.read_html(url_kosdaq, header=0, encoding="euc-kr")[0]
    df_kosdaq["시장구분"] = "KOSDAQ"

    # 합치기
    df_all = pd.concat([df_kospi, df_kosdaq], ignore_index=True)
    df_all = df_all.rename(columns={"회사명": "종목명", "종목코드": "종목코드"})
    df_all["종목코드"] = df_all["종목코드"].astype(str).str.zfill(6)

    # 야후 티커 변환 (.KS / .KQ)
    def to_yahoo(row):
        code = row["종목코드"]
        mkt = str(row["시장구분"]).upper()
        if mkt == "KOSPI":
            return f"{code}.KS"
        elif mkt == "KOSDAQ":
            return f"{code}.KQ"
        else:
            return f"{code}.KS"

    df_all["yahoo_ticker"] = df_all.apply(to_yahoo, axis=1)
    df_all["name"] = df_all["종목명"]

    return df_all[["yahoo_ticker", "name"]].drop_duplicates().reset_index(drop=True)


In [67]:
def build_universe_sp500() -> pd.DataFrame:
    """
    위키 S&P 500 구성종목 파서 (표 구조 변화/멀티헤더 방어)
    """
    import pandas as pd

    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url, header=0)
    for tbl in tables:
        # 멀티헤더면 평탄화
        if isinstance(tbl.columns, pd.MultiIndex):
            tbl.columns = [str(x[0]) for x in tbl.columns]
        else:
            tbl.columns = [str(x) for x in tbl.columns]

        cols = [c.lower() for c in tbl.columns]

        # 티커 컬럼 후보
        sym_candidates = [c for c in tbl.columns if any(k in c.lower() for k in ["symbol","ticker"])]
        if not sym_candidates:
            continue
        sym_col = sym_candidates[0]

        # 회사명 컬럼 후보
        name_candidates = [c for c in tbl.columns if any(k in c.lower() for k in ["security","company","name"])]
        name_col = name_candidates[0] if name_candidates else sym_col

        out = (tbl[[sym_col, name_col]]
               .rename(columns={sym_col: "yahoo_ticker", name_col: "name"}))
        out["yahoo_ticker"] = out["yahoo_ticker"].astype(str).str.strip()
        out["name"] = out["name"].astype(str).str.strip()
        out = out.dropna().drop_duplicates().reset_index(drop=True)

        # 최소 검증
        if "yahoo_ticker" in out.columns and len(out) > 100:
            return out

    raise RuntimeError("S&P500 테이블 파싱 실패: 열 구조가 바뀐 것 같습니다.")


In [68]:
def build_universe_nasdaq100() -> pd.DataFrame:
    """
    위키 NASDAQ-100 구성종목 파서 (표 구조 변화/멀티헤더 방어)
    """
    import pandas as pd

    url = "https://en.wikipedia.org/wiki/Nasdaq-100"
    tables = pd.read_html(url, header=0)

    for tbl in tables:
        if isinstance(tbl.columns, pd.MultiIndex):
            tbl.columns = [str(x[0]) for x in tbl.columns]
        else:
            tbl.columns = [str(x) for x in tbl.columns]

        sym_candidates = [c for c in tbl.columns if any(k in c.lower() for k in ["ticker","symbol"])]
        if not sym_candidates:
            continue
        sym_col = sym_candidates[0]

        name_candidates = [c for c in tbl.columns if any(k in c.lower() for k in ["company","name","security"])]
        name_col = name_candidates[0] if name_candidates else sym_col

        out = (tbl[[sym_col, name_col]]
               .rename(columns={sym_col: "yahoo_ticker", name_col: "name"}))
        out["yahoo_ticker"] = out["yahoo_ticker"].astype(str).str.strip()
        out["name"] = out["name"].astype(str).str.strip()
        out = out.dropna().drop_duplicates().reset_index(drop=True)

        if "yahoo_ticker" in out.columns and len(out) > 50:
            return out

    raise RuntimeError("NASDAQ-100 테이블 파싱 실패: 열 구조가 바뀐 것 같습니다.")


In [69]:
univ_parts = []
if INCLUDE_KRX:
    univ_parts.append(build_universe_krx())
if INCLUDE_SP500:
    univ_parts.append(build_universe_sp500())
if INCLUDE_NASDAQ100:
    univ_parts.append(build_universe_nasdaq100())
if EXTRA_TICKERS:
    univ_parts.append(build_universe_extra(EXTRA_TICKERS))

univ = pd.concat(univ_parts, ignore_index=True).drop_duplicates().reset_index(drop=True)
print("유니버스 총 종목 수:", len(univ))


/usr/local/lib/python3.11/dist-packages/bs4/__init__.py:339: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/bs4/__init__.py:339: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn(


유니버스 총 종목 수: 3287


##체크리스트
`univ_ok (컬럼: yahoo_ticker, name)`

####상수들:

```
RETRY, REQUEST_SLEEP, START_DATE,
MAX_WORKERS_DIV, DIV_CSV, YEARLY_CSV,
NO_DIV_CSV
```


####임포트:

```
pandas as pd, yfinance as yf, time,
datetime, from concurrent.futures import
ThreadPoolExecutor, as_completed,
from tqdm import tqdm
```



In [76]:
# ---- 야후 존재성 필터링: univ -> univ_ok ----
import yfinance as yf
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 전역 설정(이미 있다면 생략 가능)
MAX_WORKERS_CHECK = 8
REQUEST_SLEEP = 0.05
RETRY = 3

def yahoo_exists(ticker: str) -> bool:
    """티커가 야후에서 실제로 데이터가 조회되는지 빠르게 검사"""
    for _ in range(RETRY):
        try:
            # 최근 1개월 시세로 1차 체크
            hist = yf.Ticker(ticker).history(period="1mo", auto_adjust=False)
            time.sleep(REQUEST_SLEEP)
            if not hist.empty:
                return True
            # 배당 시계열로 보조 체크
            div = yf.Ticker(ticker).dividends
            time.sleep(REQUEST_SLEEP)
            if div is not None and not div.empty:
                return True
            return False
        except Exception:
            time.sleep(0.2)
    return False

# univ가 비었는지 확인
if 'univ' not in globals() or univ.empty:
    raise RuntimeError("univ가 없습니다. 유니버스 생성 셀을 먼저 실행하세요.")

# 병렬로 존재성 검사
exist_flags = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS_CHECK) as ex:
    futures = {ex.submit(yahoo_exists, t): t for t in univ["yahoo_ticker"]}
    for f in tqdm(as_completed(futures), total=len(futures), desc="야후 존재 확인"):
        t = futures[f]
        ok = False
        try:
            ok = f.result()
        except Exception:
            ok = False
        exist_flags.append((t, ok))

exist_df = pd.DataFrame(exist_flags, columns=["yahoo_ticker","exists"])

# 미존재 티커 저장(로그용)
not_found = univ.merge(exist_df, on="yahoo_ticker", how="left")
not_found = not_found[not_found["exists"] == False][["yahoo_ticker","name"]]
if len(not_found):
    not_found.to_csv(NOT_FOUND_CSV, index=False, encoding="utf-8-sig")
    print("야후 미존재 티커 수:", len(not_found), "→", NOT_FOUND_CSV)

# ✅ 여기서 최종 수집 대상 univ_ok 생성
univ_ok = univ.merge(exist_df, on="yahoo_ticker", how="left")
univ_ok = univ_ok[univ_ok["exists"] == True][["yahoo_ticker","name"]].reset_index(drop=True)

print("야후 존재 티커 수:", len(univ_ok))
try:
    display(univ_ok.head(10))
except:
    print(univ_ok.head(10))


야후 존재 확인:   0%|          | 1/3287 [00:00<34:45,  1.58it/s]ERROR:yfinance:$475230.KS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$380550.KS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$475230.KS: possibly delisted; no timezone found
ERROR:yfinance:$484120.KS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$380550.KS: possibly delisted; no timezone found
야후 존재 확인:   0%|          | 2/3287 [00:00<23:06,  2.37it/s]ERROR:yfinance:$468530.KS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$484120.KS: possibly delisted; no timezone found
ERROR:yfinance:$468530.KS: possibly delisted; no timezone found
야후 존재 확인:   0%|          | 5/3287 [00:01<08:04,  6.78it/s]ERROR:yfinance:$0041J0.KS:

야후 미존재 티커 수: 2286 → ./data/tickers_not_found.csv
야후 존재 티커 수: 1013


,yahoo_ticker,name
0,439260.KS,대한조선
1,0030R0.KS,대신밸류리츠
2,388610.KS,지에프씨생명과학
3,483650.KS,달바글로벌
4,448900.KS,한국피아이엠
5,480370.KS,씨케이솔루션
6,031210.KS,서울보증보험
7,064400.KS,LG씨엔에스
8,499790.KS,GS피앤엘
9,176750.KS,듀켐바이오


In [77]:
# 방어: 전제 확인
assert 'univ_ok' in globals(), "univ_ok가 아직 없습니다. 유니버스 생성/필터링 셀을 먼저 실행하세요."
if univ_ok.empty:
    raise RuntimeError("univ_ok가 비어 있습니다. 유니버스 생성 결과를 확인하세요.")

# START_DATE를 Timestamp로
import pandas as pd
START_TS = pd.Timestamp(START_DATE)

In [78]:
# ====== 3) 배당 수집 ======

# 날짜 비교용 Timestamp
START_TS = pd.Timestamp(START_DATE)

def fetch_dividends(ticker: str, name: str):
    """단일 티커의 최근 5년 배당 내역 수집 (list[dict] 반환)"""
    for attempt in range(RETRY):
        try:
            t = yf.Ticker(ticker)
            div = t.dividends  # pandas Series(index=Timestamp, value=float)
            time.sleep(REQUEST_SLEEP)
            if div is None or div.empty:
                return []
            # 최근 N년 필터
            div = div[div.index >= START_TS]
            return [
                {
                    "ticker": ticker,
                    "name": name,
                    "div_date": idx.strftime("%Y-%m-%d"),
                    "dividend_per_share": float(val)
                }
                for idx, val in div.items()
            ]
        except Exception as e:
            if attempt < RETRY - 1:
                time.sleep(0.5 * (attempt + 1))
            else:
                # 마지막 실패만 경고
                print(f"[WARN] dividends fetch fail: {ticker} ({name}) → {e}")
                return []

# 병렬 수집
assert 'univ_ok' in globals() and not univ_ok.empty, "univ_ok가 없거나 비었습니다. 유니버스 필터링을 먼저 실행하세요."
all_rows = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS_DIV) as ex:
    futures = {
        ex.submit(fetch_dividends, r["yahoo_ticker"], r["name"]): (r["yahoo_ticker"], r["name"])
        for _, r in univ_ok.iterrows()
    }
    for f in tqdm(as_completed(futures), total=len(futures), desc="배당 수집"):
        try:
            rows = f.result()
            if rows:
                all_rows.extend(rows)
        except Exception as e:
            tick, nm = futures[f]
            print(f"[WARN] future failed: {tick} ({nm}) → {e}")

# DataFrame 변환/정리
df_div = pd.DataFrame(all_rows, columns=["ticker", "name", "div_date", "dividend_per_share"])
if not df_div.empty:
    df_div = df_div.sort_values(["ticker", "div_date"]).drop_duplicates().reset_index(drop=True)

# 최근 5년 배당 전무한 유효 티커 기록
no_div_tickers = set(univ_ok["yahoo_ticker"]) - set(df_div["ticker"])
print("최근 5년 배당 없음 티커 수:", len(no_div_tickers))
if no_div_tickers:
    univ_ok[univ_ok["yahoo_ticker"].isin(no_div_tickers)] \
        .to_csv(NO_DIV_CSV, index=False, encoding="utf-8-sig")
    print("→ 목록 저장:", NO_DIV_CSV)

# ====== 4) 저장 & 연도 합계 ======
df_div["collected_at"] = datetime.now().strftime("%Y-%m-%d %H:%M")
df_div.to_csv(DIV_CSV, index=False, encoding="utf-8-sig")
print("일자별 배당 저장:", DIV_CSV, "행수:", len(df_div))

if not df_div.empty:
    df_div["year"] = df_div["div_date"].str[:4]
    yearly = (df_div.groupby(["ticker","name","year"], as_index=False)["dividend_per_share"]
              .sum()
              .rename(columns={"dividend_per_share":"dividend_sum_per_share"}))
    yearly.to_csv(YEARLY_CSV, index=False, encoding="utf-8-sig")
    print("연도별 합계 저장:", YEARLY_CSV, "행수:", len(yearly))
else:
    print("연도별 합계 생성 스킵(배당 데이터 없음)")


배당 수집:   0%|          | 3/1013 [00:01<07:47,  2.16it/s]

[WARN] dividends fetch fail: 439260.KS (대한조선) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 483650.KS (달바글로벌) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 448900.KS (한국피아이엠) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 0030R0.KS (대신밸류리츠) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 031210.KS (서울보증보험) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 480370.KS (씨케이솔루션) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 388610.KS (지에프씨생명과학) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 064400.KS (LG씨엔에스) → Too Many Requests. Rate limited. Try after a while.


배당 수집:   1%|          | 12/1013 [00:03<03:51,  4.33it/s]

[WARN] dividends fetch fail: 499790.KS (GS피앤엘) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 176750.KS (듀켐바이오) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 044990.KS (에이치엔에스하이텍) → Too Many Requests. Rate limited. Try after a while.


배당 수집:   1%|▏         | 15/1013 [00:03<02:33,  6.52it/s]

...

[WARN] dividends fetch fail: 224760.KS (엔에스컴퍼니) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 214430.KS (아이쓰리시스템) → Too Many Requests. Rate limited. Try after a while.


배당 수집: 100%|█████████▉| 1011/1013 [03:28<00:00,  6.66it/s]

[WARN] dividends fetch fail: 218150.KS (미래생명자원) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: 003200.KS (일신방직) → Too Many Requests. Rate limited. Try after a while.


배당 수집: 100%|██████████| 1013/1013 [03:29<00:00,  4.84it/s]

[WARN] dividends fetch fail: EQIX (Equinix) → Too Many Requests. Rate limited. Try after a while.
[WARN] dividends fetch fail: NXPI (NXP Semiconductors) → Too Many Requests. Rate limited. Try after a while.
최근 5년 배당 없음 티커 수: 1013
→ 목록 저장: ./data/tickers_with_no_dividends.csv
일자별 배당 저장: ./data/dividends_all_companies_5y.csv 행수: 0
연도별 합계 생성 스킵(배당 데이터 없음)


##참고/주의

야후 데이터 특성: 일부 종목은 배당 기록이 늦게 반영되거나 누락될 수 있어요. 중요한 종목은 KRX/공시 데이터로 교차검증 권장.

속도 조절: MAX_WORKERS_*와 REQUEST_SLEEP로 조절하세요. 너무 올리면 레이트리밋 위험.

해외 추가: 도요타 7203.T, 토요타 ADR TM, 런던 상장 HSBA.L 같은 거래소 접미사가 필요한 종목은 EXTRA_TICKERS에 직접 넣으세요.

범위 변경: 최근 3년/10년으로 바꾸려면 LAST_N_YEARS만 수정하면 됩니다.

Colab이 위키를 읽다가 가끔 네트워크 이슈를 냅니다. 그럴 땐 해당 셀만 한 번 더 실행하세요.

여전히 안 된다면, 위키 대신 공식 리스트 CSV(예: S&P500 tickers 깃허브 미러)로 대체하는 백업 경로를 붙일 수 있어요